In [1]:
import pandas as pd
import numpy as np

/home/ibs/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [113]:
dataset = pd.read_csv('Datasets/Products.csv', lineterminator='\n')

In [114]:
dataset.count(), dataset.dropna().count(), dataset["price"].map(lambda x: x[0]).unique(), dataset["price"].map(lambda x: x[-3:]).unique()

(Unnamed: 0             7156
 id                     7156
 product_name           7156
 category               7156
 product_description    7156
 price                  7156
 location               7156
 dtype: int64,
 Unnamed: 0             7156
 id                     7156
 product_name           7156
 category               7156
 product_description    7156
 price                  7156
 location               7156
 dtype: int64,
 array(['£'], dtype=object),
 array(['.00', '.99', '.78', '.01', '.97', '.25', '.50', '.20', '.90',
        '.80', '.60', '.23', '.05', '.75', '.56', '.40', '.44', '.95',
        '.66', '.35', '.85', '.30', '.45', '.16', '.69', '.49', '.55',
        '.09', '.11'], dtype=object))

# So the dataset seems to already be clean

Now we convert the price column

In [115]:
dataset_cleaned = dataset

def remove_pound_sign(string_to_replace) -> str:
    return 

dataset_cleaned['price'] = dataset_cleaned['price'].map(lambda x : x.replace('£', '')) #Removes the pound signs
dataset_cleaned['price'] = dataset_cleaned['price'].map(lambda x: x.replace(',', '')) #Removes commas

dataset_cleaned['price'] = pd.to_numeric(dataset_cleaned['price'])

# Now we extract the root category from each item

In [116]:
dataset_cleaned['category'] = dataset_cleaned['category'].map(lambda x: x.split(' /' )[0])

In [117]:
dataset_cleaned['category'].unique()

array(['Home & Garden', 'Baby & Kids Stuff', 'DIY Tools & Materials',
       'Music, Films, Books & Games', 'Phones, Mobile Phones & Telecoms',
       'Clothes, Footwear & Accessories', 'Other Goods',
       'Health & Beauty', 'Sports, Leisure & Travel', 'Appliances',
       'Computers & Software', 'Office Furniture & Equipment',
       'Video Games & Consoles'], dtype=object)

Now we create an encoder to go to and from the categories and integers

In [118]:
list_of_categories = list(dataset_cleaned['category'].unique())

encoder = {x: list_of_categories.index(x) for x in list_of_categories}

decoder = {list_of_categories.index(x):x for x in list_of_categories}

encoder, decoder

({'Home & Garden': 0,
  'Baby & Kids Stuff': 1,
  'DIY Tools & Materials': 2,
  'Music, Films, Books & Games': 3,
  'Phones, Mobile Phones & Telecoms': 4,
  'Clothes, Footwear & Accessories': 5,
  'Other Goods': 6,
  'Health & Beauty': 7,
  'Sports, Leisure & Travel': 8,
  'Appliances': 9,
  'Computers & Software': 10,
  'Office Furniture & Equipment': 11,
  'Video Games & Consoles': 12},
 {0: 'Home & Garden',
  1: 'Baby & Kids Stuff',
  2: 'DIY Tools & Materials',
  3: 'Music, Films, Books & Games',
  4: 'Phones, Mobile Phones & Telecoms',
  5: 'Clothes, Footwear & Accessories',
  6: 'Other Goods',
  7: 'Health & Beauty',
  8: 'Sports, Leisure & Travel',
  9: 'Appliances',
  10: 'Computers & Software',
  11: 'Office Furniture & Equipment',
  12: 'Video Games & Consoles'})

# Now we merge this with the original table:

In [119]:
dataset_cleaned['label_index'] = dataset_cleaned['category'].map(lambda x:encoder[x])

# Next we merge this with the images table:

In [127]:
images_dataset = pd.read_csv('Datasets/Images.csv')

dataset_cleaned["merge_column"] = dataset_cleaned['id']
images_dataset['merge_column'] = images_dataset['product_id']

dataset_cleaned.dtypes, images_dataset.dtypes

(Unnamed: 0               int64
 id                      object
 product_name            object
 category                object
 product_description     object
 price                  float64
 location                object
 label_index              int64
 merge_column            object
 dtype: object,
 Unnamed: 0       int64
 id              object
 product_id      object
 merge_column    object
 dtype: object)

In [135]:
merged_df = images_dataset.merge(dataset_cleaned, on='merge_column')

In [136]:
merged_df = merged_df[["id_x", "product_id", "label_index"]]
merged_df

,id_x,product_id,label_index
0,912bb259-3ad9-457b-9db1-ce1da9016057,5f5f57d7-778f-4336-bb10-b43863418c8c,0
1,b166d305-b852-4bdd-83f4-465b20da94fa,5f5f57d7-778f-4336-bb10-b43863418c8c,0
2,68f5a29d-0075-4d60-81c1-ab684a82e50c,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,0
3,f6a309d7-d247-446a-9b5e-aceefdd4334d,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,0
4,2c2b3a6f-15b3-4289-937a-15482d9f5781,8292aa4e-7f1b-4655-bf0e-f1f2c9e3ffaf,0
...,...,...,...
12599,cdec1c5c-c4b1-42db-afbe-3fa68ea4b87d,2b0a652b-46a2-4297-b619-5efeeb222787,12
12600,dc99e40f-6b15-494d-9fb7-f0d02e9781f9,719fd40a-870e-4144-b324-55dff2e66fb4,12
12601,c8488028-bf07-4258-a4c2-56d2fe387835,719fd40a-870e-4144-b324-55dff2e66fb4,12
12602,c6113145-89c8-47cd-9211-38f29d016cc7,86d1806b-5575-4a7e-9160-f24f12be6c95,12


In [139]:
merged_df = merged_df.rename(columns={'id_x':'id'})
merged_df

,id,product_id,label_index
0,912bb259-3ad9-457b-9db1-ce1da9016057,5f5f57d7-778f-4336-bb10-b43863418c8c,0
1,b166d305-b852-4bdd-83f4-465b20da94fa,5f5f57d7-778f-4336-bb10-b43863418c8c,0
2,68f5a29d-0075-4d60-81c1-ab684a82e50c,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,0
3,f6a309d7-d247-446a-9b5e-aceefdd4334d,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,0
4,2c2b3a6f-15b3-4289-937a-15482d9f5781,8292aa4e-7f1b-4655-bf0e-f1f2c9e3ffaf,0
...,...,...,...
12599,cdec1c5c-c4b1-42db-afbe-3fa68ea4b87d,2b0a652b-46a2-4297-b619-5efeeb222787,12
12600,dc99e40f-6b15-494d-9fb7-f0d02e9781f9,719fd40a-870e-4144-b324-55dff2e66fb4,12
12601,c8488028-bf07-4258-a4c2-56d2fe387835,719fd40a-870e-4144-b324-55dff2e66fb4,12
12602,c6113145-89c8-47cd-9211-38f29d016cc7,86d1806b-5575-4a7e-9160-f24f12be6c95,12


In [140]:
merged_df.to_csv('Datasets/training_data.csv')

# Clean Image Dataset:

In [11]:
from PIL import Image
import os

def resize_image(final_size, im):
    size = im.size
    ratio = float(final_size) / max(size)
    new_image_size = tuple([int(x*ratio) for x in size])
    im = im.resize(new_image_size)
    new_im = Image.new("RGB", (final_size, final_size))
    new_im.paste(im, ((final_size-new_image_size[0])//2, (final_size-new_image_size[1])//2))
    return new_im

def clean_images(path_to_extract = "Datasets/images/", path_to_save = "Datasets/cleaned_images/"):
    dirs = os.listdir(path_to_extract)
    final_size = 512
    for n, item in enumerate(dirs, 1):
        #print(n, item)
        im = Image.open(path_to_extract + item)
        #print(im.width, im.height)
        new_im = resize_image(final_size, im)
        #print(new_im.width, new_im.height)
        new_im.save(path_to_save + item)


In [12]:
clean_images()